# 第29章 运算符重载

- “运算符重载”只是意味着在类方法中拦截内置的操作——当类的实例出现在内置操作中，Python自动调用你的方法，并且你的方法的返回值变成了相应操作的结果
- 类可重载所有Python表达式运算符
- 类也可重载打印、函数调用、属性点号运算等内置运算
- 重载使类实例的行为像内置类型
- 重载使通过提供特殊名称的类方法来实现
- 运算符重载方法并非必需的，并且通常也不是默认的，如果你没有编写或继承一个运算符重载方法，只是意味着你的类不会支持相应的操作

In [1]:
class Number:
    def __init__(self, start):
        self.data = start
    def __sub__(self, other): # 减法操作
        return Number(self.data - other)

X = Number(5)
Y = X - 2
Y.data

3

## 常见的运算符重载方法

- 所有重载方法的名称前后都有两个下划线字符，以便把同类中定义的变量名区别开来，特殊方法名称和表达式或运算的映射关系，是由Python语言预先定义好的
- 如果没有定义运算符重载方法的话，它可能继承自超类

### 索引和分片：`__getitem__`和`__setitem__`

- 当定义了`__getitem__`方法，出现`X[i]`这样的索引运算时，Python会把`X`作为第一个参数传递，把`i`作为第二个参数传递

In [2]:
class Indexer:
    def __getitem__(self, index):
        return index ** 2
    
X = Indexer()
X[2]

4

### 拦截分片

- 对于分片表达式，也调用`__getitem__`，正式地讲，内置类型以同样的方式处理分片
- 事实上，分片边界绑定到了一个分片对象中，并且传递给索引的列表实现

In [3]:
data = [5, 6, 7, 8, 9]
print slice(2, 4)
print data[slice(2, 4)]

slice(2, 4, None)
[7, 8]


In [4]:
class Indexer:
    data = [5, 6, 7, 8, 9]
    def __getitem__(self, index):
        print 'getitem: ', index
        print self.data[index]
    
X = Indexer()
X[0]
X[-1]
X[2:4]

getitem:  0
5
getitem:  -1
9
getitem:  slice(2, 4, None)
[7, 8]


- `__setitem__`索引赋值方法类似地拦截索引和分片赋值——它为后者接收了一个分片对象

In [5]:
def __setitem__(self, index, value):
    self.data[index] = value

### 索引迭代：`__getitem__`

In [6]:
class stepper:
    def __getitem__(self, i):
        return self.data[i]
X = stepper()
X.data = 'Spam'
print X[1]
for item in X:
    print item

p
S
p
a
m


- 任何支持for循环的类也会自动支持Python所有迭代环境

In [7]:
'p' in X

True

In [8]:
[c for c in X]

['S', 'p', 'a', 'm']

In [9]:
map(str.upper, X)

['S', 'P', 'A', 'M']

In [10]:
(a, b, c, d) = X
a, c, d

('S', 'a', 'm')

### 迭代器对象：`__iter__`和`__next__`

- 尽管上一节的`__getitem__`有效，但它真的只是迭代的一种退而求其次的方法，如今Python中所有的迭代环境都会先尝试`__iter__`方法，再尝试`__getitem__`，也即，只有在对象不支持迭代协议的时候，才会尝试索引运算；一般来讲，你也应该优先使用`__iter__`，它能够比`__getitem__`更好地支持一般的迭代环境
- 从技术角度来讲，迭代环境是通过调用内置函数iter去尝试寻找`__iter__`方法来实现的，这个方法会返回一个迭代器对象，Python会重复调用这个迭代器对象的next方法，知道发生StopIteration异常；否则，Python会改用`__getitem__`机制，通过偏移量重复索引，直到引发IndexError异常
- 注意，在Python2.6中，`__next__`改为`next`

### 用户定义的迭代器

In [11]:
class Squares:
    def __init__(self, start, stop):
        self.value = start - 1
        self.stop = stop
    def __iter__(self):
        return self
    def next(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2
for i in Squares(1, 5):
    print i

1
4
9
16
25


In [12]:
X = Squares(1, 5)
I = iter(X)
next(I)

1

- 相较于`__getitem__`，有时候`__iter__`会更难用，例如，它不能用来索引

In [13]:
X = Squares(1, 5)
# 下列索引运算会报错
# X[1]

- `__iter__`支持成员关系测试、类型构造函数、序列赋值运算等

In [14]:
[n for n in Squares(1, 5)]

[1, 4, 9, 16, 25]

In [15]:
4 in Squares(1, 5)

True

### 有多个迭代器的对象

- 当我们用类编写用户定义的迭代器的时候，由我们来决定是支持一个单个的或是多个活跃的迭代，要达到多个迭代器的效果，`__iter__`只需替迭代器定义新的定义新的状态对象，而不是返回self

In [16]:
class SkipObject:
    def __init__(self, wrapped):
        self.wrapped = wrapped
    def __iter__(self):
        return SkipIterator(self.wrapped)
    
class SkipIterator:
    def __init__(self, wrapped):
        self.wrapped = wrapped
        self.offset = 0
    def next(self):
        if self.offset >= len(self.wrapped):
            raise StopIteration
        else:
            item = self.wrapped[self.offset]
            self.offset += 2
            return item
        
alpha = 'abcdef'
skipper = SkipObject(alpha)
I = iter(skipper)
print next(I), next(I), next(I)

# 每个循环都会获得独立的迭代器对象来记录自己的状态信息
for x in skipper:
    for y in skipper:
        print x + y

a c e
aa
ac
ae
ca
cc
ce
ea
ec
ee


- 迭代器是很强大的工具，可让我们把任意对象的外观和用法变得很像其它序列和可迭代对象

### 成员关系：`__contains__`、`__iter__`和`__getitem__`

- 运算符重载往往是多个层级的：类可以提供特定的方法，或者用作退而求其次选项的更通用的替代方法

- 在迭代领域，类通常把in成员关系运算符实现为一个迭代，使用`__iter__`方法或`__getitem__`方法，要支持更加特定的成员关系，类可能编写一个`__contains__`方法——当出现的时候，该方法优先于`__iter__`方法，`__iter__`方法优先于`__getitem__`方法

In [17]:
from __future__ import print_function
class Iters:
    def __init__(self, value):
        self.data = value
    def __getitem__(self, i):
        print("get[%s]:" % i, end='')
        return self.data[i]
    def __iter__(self):
        print("iter=> ", end='')
        self.ix = 0
        return self
    def next(self):
        print('next:', end='')
        if self.ix == len(self.data): 
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item
    def __contains__(self, x):
        print('contains: ', end='')
        return x in self.data
    
X = Iters([1, 2, 3, 4, 5])
print(3 in X)
for i in X:
    print(i, end=' | ')

print()
print([i **2 for i in X]) # __iter__函数每次会执行self.ix = 0，所以可以迭代多次
print(map(bin, X))

I = iter(X)
while True:
    try:
        print(next(I), end='@')
    except StopIteration:
        break

contains: True
iter=> next:1 | next:2 | next:3 | next:4 | next:5 | next:
iter=> next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter=> next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']
iter=> next:1@next:2@next:3@next:4@next:5@next:

- 如果注释掉`__contains__`，成员关系将会路由到通用的`__iter__`

In [18]:
class Iters:
    def __init__(self, value):
        self.data = value
    def __getitem__(self, i):
        print("get[%s]:" % i, end='')
        return self.data[i]
    def __iter__(self):
        print("iter=> ", end='')
        self.ix = 0
        return self
    def next(self):
        print('next:', end='')
        if self.ix == len(self.data): 
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item
    #def __contains__(self, x):
    #    print('contains: ', end='')
    #    return x in self.data
    
X = Iters([1, 2, 3, 4, 5])
print(3 in X)
for i in X:
    print(i, end=' | ')

print()
print([i **2 for i in X]) # __iter__函数每次会执行self.ix = 0，所以可以迭代多次
print(map(bin, X))

I = iter(X)
while True:
    try:
        print(next(I), end='@')
    except StopIteration:
        break

iter=> next:next:next:True
iter=> next:1 | next:2 | next:3 | next:4 | next:5 | next:
iter=> next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter=> next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']
iter=> next:1@next:2@next:3@next:4@next:5@next:

- 如果注释掉`__contains__`和`__iter__`，索引`__getitem__`替代方法会被调用

In [19]:
class Iters:
    def __init__(self, value):
        self.data = value
    def __getitem__(self, i):
        print("get[%s]:" % i, end='')
        return self.data[i]
    #def __iter__(self):
    #    print("iter=> ", end='')
    #    self.ix = 0
    #    return self
    def next(self):
        print('next:', end='')
        if self.ix == len(self.data): 
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item
    #def __contains__(self, x):
    #    print('contains: ', end='')
    #    return x in self.data
    
X = Iters([1, 2, 3, 4, 5])
print(3 in X)
for i in X:
    print(i, end=' | ')

print()
print([i **2 for i in X]) # __iter__函数每次会执行self.ix = 0，所以可以迭代多次
print(map(bin, X))

I = iter(X)
while True:
    try:
        print(next(I), end='@')
    except StopIteration:
        break

get[0]:get[1]:get[2]:True
get[0]:1 | get[1]:2 | get[2]:3 | get[3]:4 | get[4]:5 | get[5]:
get[0]:get[1]:get[2]:get[3]:get[4]:get[5]:[1, 4, 9, 16, 25]
get[0]:get[1]:get[2]:get[3]:get[4]:get[5]:['0b1', '0b10', '0b11', '0b100', '0b101']
get[0]:1@get[1]:2@get[2]:3@get[3]:4@get[4]:5@get[5]:

- `__getitem__`方法甚至更加通用，除了迭代，它还拦截显式索引和分片

In [20]:
X = Iters('spam')
X[0]

get[0]:

's'

In [21]:
X[1:3]

get[slice(1, 3, None)]:

'pa'

### 属性引用：`__getattr__`和`__setattr__`

- `__getattr__`方法是拦截属性点号运算，更确切的说，当通过对未定义（不存在）属性名称和实例进行点号运算时，就会用属性名称作为字符串调用这个方法，如果Python可以通过继承树搜索流程找到这个属性，该方法就不会被调用

In [22]:
class empty:
    def __getattr__(self, attrname):
        if attrname == "age":
            return 40
        else:
            raise AttributeError, attrname

X = empty()
X.age
# 以下语句会报错
#X.name

40

- `__setattr__`会拦截所有属性的赋值语句，如果定义了这个方法，`self.attr = value`会变成`self.__setattr__('attr', value)`，由于在`__setattr__`中对任何self属性做赋值，都会再调用`__setattr__`，导致无穷递归循环，因此内部要使用`self.__dict__['name'] = x`来赋值

In [23]:
class accesscontrol:
    def __setattr__(self, attr, value):
        if attr == 'age':
            self.__dict__[attr] = value
        else:
            raise AttributeError, attr + 'not allowed'
    
X = accesscontrol()
X.age = 40
X.age
# 以下语句会报错
#X.name = 'mel'

40

### `__repr__`和`__str__`会返回字符串表达形式

- `__repr__`比`__str__`更加通用，而`__str__`只是在某些场景下会被调用，因此，如果想让所有环境都有统一的显示，`__repr__`是最佳选择

### 右侧加法和原处加法：`__radd__`和`__iadd__`

- 之前讲过的`__add__`方法并不支持`+`运算符右侧使用实例对象，要实现这类表达式，而支持可互换的运算符，可以一并编写`__radd__`方法，只有当右侧的对象是类实例，而左边对象不是类实例时，Python才会调用`__radd__`，否则由左侧对象调用`__add__`

In [24]:
class Commuter:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val
    
x = Commuter(88)
y = Commuter(99)
x + 1
1 + y
x + y # __add__触发了__radd__

add 88 1
radd 99 1
add 88 <__main__.Commuter instance at 0x10e1ffef0>
radd 99 88


187

- 为了也实现+=原处扩展相加，编写一个`__iadd__`或`__add__`，如果前者空缺，则使用后者

In [25]:
class Number:
    def __init__(self, val):
        self.val = val
    def __iadd__(self, other):
        self.val += other
        return self
x = Number(5)
x += 1
x += 1
x.val

7

- 每个二元运算都有类似的右侧和原处重载方法，它们以相同的方式工作（例如，`__mul__`、`__rmul__`和`__imul__`）；右侧方法是一个高级话题，并且实际中很少用到，只有在需要运算符具有交换性的时候，才会编写它们

### Call表达式：`__call__`

- 如果定义了，Python就会为实例应用函数调用表达式运行`__call__`方法，这样可以让类实例的外观和用法类似于函数

In [26]:
class Callee:
    def __call__(self, *pargs, **kargs):
        print ('Called: ', pargs, kargs)

C = Callee()
C(1, 2, 3)
C(1, 2, 3, x=4, y=5)

Called:  (1, 2, 3) {}
Called:  (1, 2, 3) {'y': 5, 'x': 4}


- 更准确地说，我们在第18章介绍的所有参数传递方式，`__call__`方法都支持

In [27]:
class Prod:
    def __init__(self, value):
        self.value = value
    def __call__(self, other):
        return self.value * other

x = Prod(2)
x(3)

6

### 比较：`__lt__`、`__gt__`和其它方法

In [28]:
class C:
    data = 'spam'
    def __gt__(self, other):
        return self.data > other
    def __lt__(self, other):
        return self.data < other

X = C()
print (X > 'ham') # runs __gt__
print (X < 'ham') # runs __lt__

True
False


- 在Python2.6中，如果没有定义更加具体的方法的话，`__cmp__`作为一种退而求其次的方法：它返回一个小于、等于或大于0的数，以表示比较其两个参数的结果，这个方法往往使用cmp(x, y)内置函数来计算其结果

In [29]:
class C:
    data = 'spam'
    def __cmp__(self, other):
        return cmp(self.data, other)
        
X = C()
print (X > 'ham')
print (X < 'ham')

True
False


### 布尔测试：`__bool__`和`__len__`

- 在布尔环境中，Python首先尝试`__bool__`来获取一个直接的布尔值，若没定义，则尝试`__len__`，它会根据对象的长度确定一个真值

In [30]:
class Truth:
    def __bool__(self):
        return True

X = Truth()
if X:
    print('yes')

yes


In [31]:
class Truth:
    def __len__(self):
        return 0

X = Truth()
if not X:
    print('no')

no


In [32]:
# python2.6 会首先尝试__len__
# python3.0 会首先尝试__bool__
class Truth:
    def __bool__(self):
        return True
    def __len__(self):
        return 0
    
X = Truth()
if not X:
    print('yes')

yes


- 如果没有定义以上两个方法，则对象为真

In [33]:
class Truth:
    pass

X = Truth()
bool(X)

True

### 对象析构函数：`__del__`

- 当实例空间被收回时（垃圾收集），析构函数会自动执行

In [34]:
class Life:
    def __init__(self, name='unknown'):
        print('Hello', name)
        self.name = name
    def __del__(self):
        print('Goodbye', self.name)
        
brian = Life('Brian')

Hello Brian


In [35]:
brian = 'loretta'

Goodbye Brian


- 基于某些原因，析构函数在Python中用的不多